## This notebook is used to analyse collision warning results

In [ ]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
font = {'family' : 'Arial',
        'size'   : 9}
plt.rc('font', **font)
plt.rcParams['mathtext.fontset'] = 'stix' #dejavuserif
sys.path.append('../../')
from Demonstration.demonstration_utils import *

def to_grayscale(fig):
    fig.canvas.draw()
    img = np.array(fig.canvas.renderer.buffer_rgba())
    grayscale_img = np.dot(img[..., :3], [0.2989, 0.5870, 0.1140])
    fig_gray, ax_gray = plt.subplots(figsize=(fig.get_size_inches()), dpi=fig.dpi)
    ax_gray.imshow(grayscale_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=255)
    ax_gray.axis('off')  # Turn off the axis
    ax_gray.set_title('Grayscale plot')

path_matched_100Car = '../../../Process_100Car/MatchedEvents/'  # Path to processed data outside the repository
path_processed = '../../Data/ProcessedData/100Car/'
path_input = '../../Data/InputData/'
path_output = '../../Data/OutputData/'
fig_path = r'C:/SURFdrive/PhD progress/PhDResearch/4_Conflict/AMAR/Figures/'

### Originally matched events

In [ ]:
meta_crashes = pd.read_csv(path_matched_100Car+'HundredCar_metadata_Crashes.csv')
meta_crashes.groupby('target').size()

In [ ]:
meta_nearcrashes = pd.read_csv(path_matched_100Car+'HundredCar_metadata_NearCrashes.csv')
meta_nearcrashes.groupby('target').size().sort_values(ascending=False)

### Selected events

In [ ]:
events = pd.read_hdf(path_input + '100Car/HundredCar_Crashes.h5', key='data')
meta = pd.read_csv(path_input + '100Car/HundredCar_metadata_Crashes.csv').set_index('webfileid')
print('Crash targets: ', meta.groupby('target').size())

In [ ]:
events = pd.read_hdf(path_input + '100Car/HundredCar_NearCrashes.h5', key='data')
meta = pd.read_csv(path_input + '100Car/HundredCar_metadata_NearCrashes.csv').set_index('webfileid')
print('Crash targets: ', meta.groupby('target').size().sort_values(ascending=False))
print('Near-crash with veh in adjacent lane: ', meta[meta['target']=='vehicle in adjacent lane'].index.values)

### Warning evaluation

In [ ]:
ttc_warning, statistics_ttc, optimal_ttc = read_data('TTC', path_output)
drac_warning, statistics_drac, optimal_drac = read_data('DRAC', path_output)
psd_warning, statistics_psd, optimal_psd = read_data('PSD', path_output)
unified_warning, statistics_unified, optimal_unified = read_data('Unified', path_output)

In [ ]:
ttc_selected = read_selected('TTC', path_output)
drac_selected = read_selected('DRAC', path_output)
psd_selected = read_selected('PSD', path_output)
unified_selected = read_selected('Unified', path_output)

In [ ]:
fig = plot_warning([statistics_ttc, optimal_ttc, ttc_selected],
                   [statistics_drac, optimal_drac, drac_selected],
                   [statistics_psd, optimal_psd, psd_selected],
                   [statistics_unified, optimal_unified, unified_selected])
to_grayscale(fig)

In [ ]:
fig.savefig(fig_path + 'warning_evaluation.pdf', bbox_inches='tight', dpi=600)

In [ ]:
statistics_unified[(statistics_unified['true positive rate']>0.9)&(statistics_unified['false positive rate']<0.05)]

### Summary of results

In [ ]:
for unified, tof_unified in zip([optimal_unified['true warning'], ~optimal_unified['true warning']], ['unified true', 'unified false']):
    for ttc, tof_ttc in zip([optimal_ttc['true warning'], ~optimal_ttc['true warning']], ['ttc true', 'ttc false']):
        for drac, tof_drac in zip([optimal_drac['true warning'], ~optimal_drac['true warning']], ['drac true', 'drac false']):
            statistics = optimal_ttc[ttc&drac&unified]
            print(tof_unified, ',', tof_ttc, ',', tof_drac, ',', len(statistics), ',', statistics.index.values)

In [ ]:
print('unified false warning:', optimal_unified[optimal_unified['false warning']].index.values)